In [1]:
import glob
import os
import pathlib

import numpy as np
import pandas as pd

from util.image import unnormalize

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import functional as F

from torchvision import transforms
from torchinfo import torchinfo
from tqdm import tqdm
import matplotlib.pyplot as plt

import albumentations as A
import torch.nn.functional as F

from PIL import Image

import torchmetrics
from torchvision.utils import save_image, make_grid
import cv2
from util.io import load_ckpt

from util.loss import  InpaintingLoss
import os, glob

from networks import CNN, EfficientUNet


device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")

c:\ProgramData\Anaconda3\envs\cvproject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
FACERECOG_PATH = 'mask_model.pth'
INFILLER_PATH  = 'efficientunet_temp.pth'


def load_file(path):
    with open(path, 'rb') as f:
        return torch.load(f)

In [6]:
# loading the mask-detection model

model = CNN().to(device)
model.load_state_dict(load_file(FACERECOG_PATH))

<All keys matched successfully>

In [ ]:
face_img_inference: 

## Masked

In [14]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
nose_cascade = cv2.CascadeClassifier( cv2.data.haarcascades + 'haarcascade_mcs_nose.xml')

def getMask(img):
  # Load the image
  # img = cv2.imread(imagePath)
  # Convert the image to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # Detect faces
  faces = face_cascade.detectMultiScale(gray, 1.3, 5)
  # Loop through each face and detect eyes

  # Create a mask image
  mask = np.ones(img.shape[:2], np.uint8)
  
  facesCount = 0
  for (x,y,w,h) in faces:

      roi_gray = gray[y:y+h, x:x+w]
      roi_color = img[y:y+h, x:x+w]
      eyes = eye_cascade.detectMultiScale(roi_gray)
      
      # Loop through each eye and get the coordinates
      # eye_y = img.shape[0]
      cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)        # create bounding box for the face
      eyeCount = len(eyes)
      lowesty = 0
      for (ex,ey,ew,eh) in eyes:
          eye_x = x + ex + int(ew/2)
          eye_y = y + ey + int(eh/2)  
          lowesty = max(lowesty, eye_y + eh//4)
        #   cv2.rectangle(img, (eye_x-ew//2, eye_y-eh//2), (eye_x+ew//2, eye_y+eh//2), (0, 0, 255), 4)
        
      face_img = img[y:y+h, x:x+w]


        
        # Display the cropped face image
      
      if(eyeCount >= 1 and True):
        facesCount += 1
        # cv2.imshow(face_img)
        # cv2.waitKey(0)
        for posx in range(x, x+w):
          for posy in range(lowesty, min(y+h+h//12, mask.shape[0])):
            mask[posy][posx] = 0
      if(facesCount > 0):
        break

  masked_image = cv2.bitwise_and(img, img, mask = mask) 
  return masked_image


cap = cv2.VideoCapture(0)
while True:
    # Read the frame
    _, img = cap.read()
    gray = getMask(img)
    cv2.imshow('img', gray)
    if cv2.waitKey(0):
        break


cap.release()
cv2.destroyAllWindows()

## Unmasked

In [ ]:
# import cv2
# import numpy as np
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
# nose_cascade = cv2.CascadeClassifier( cv2.data.haarcascades + 'haarcascade_mcs_nose.xml')

# def getMask(img):
#   # Load the image
#   # img = cv2.imread(imagePath)
#   # Convert the image to grayscale
#   gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#   # Detect faces
#   faces = face_cascade.detectMultiScale(gray, 1.3, 5)
#   # Loop through each face and detect eyes

#   # Create a mask image
#   mask = np.ones(img.shape[:2], np.uint8)
#   for (x,y,w,h) in faces:
#       roi_gray = gray[y:y+h, x:x+w]
#       roi_color = img[y:y+h, x:x+w]
#       eyes = eye_cascade.detectMultiScale(roi_gray)
      
#       # Loop through each eye and get the coordinates
#       # eye_y = img.shape[0]
#       cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)
#       eyeCount = len(eyes)
#       lowesty = 0
#       for (ex,ey,ew,eh) in eyes:
#           eye_x = x + ex + int(ew/2)
#           eye_y = y + ey + int(eh/2)  
#           lowesty = max(lowesty, eye_y + eh//3)
#           cv2.rectangle(img, (eye_x-ew//2, eye_y-eh//2), (eye_x+ew//2, eye_y+eh//2), (0, 0, 255), 4)
        
#       if(eyeCount >= 1 and True):
#         for posx in range(x, x+w):
#           for posy in range(lowesty, y+h):
#             mask[posy][posx] = 0
#   masked_image = cv2.bitwise_and(img, img, mask=mask)
#   # Display the image
#   return masked_image


# cap = cv2.VideoCapture(0)
# while True:
#     # Read the frame
#     _, img = cap.read()
#     gray = getMask(img)
#     cv2.imshow('img', gray)
#     k = cv2.waitKey(30) & 0xff
#     if k==27:
#         break


# cap.release()
# cv2.destroyAllWindows()

KeyboardInterrupt: 